In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

import pickle
import random
from string import punctuation

import gensim
from gensim import corpora

import spacy
from spacy.lang.en import English

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katieta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/katieta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# tokenizing/lemma-tizing stuff

parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
#         elif token.orth_.startswith('#'):
#             lda_tokens.append('HASHTAG')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

def get_text_data(corpus, token_len=20):
    text_data = []
    for i in range(len(corpus)):
        if i % 20000 == 0:
            print("{} / {}".format(i, len(corpus)))
#             if i > 0:
#                 print(text_data[i - 1])
        try:
            tokens = prepare_text_for_lda(corpus[i])
            while len(tokens) < token_len:
                tokens.append('STOP')
                
            if len(tokens) > token_len:
                tokens = tokens[:token_len - 1]
                tokens.append('STOP')
                text_data.append(tokens)
            else:
                text_data.append(tokens)
        except Exception as e:
            print(e)
    print("finished")
    return text_data

# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1
# print(stoplist)
en_stop = set(nltk.corpus.stopwords.words('english'))
# print(stoplist)

def get_dict_corpus(text_data):
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2idx(text) for text in text_data]
    return dictionary, corpus

In [4]:
dem_tweets = pd.read_csv('../data/tweet_data/dem_tweets_cleaned.csv', index_col=False) # dem tweets w/ uncontested candidates removed
dem_tweets['party'] = [0 for i in range(len(dem_tweets))]
rep_tweets =pd.read_csv('../data/tweet_data/rep_tweets_cleaned.csv', index_col=False) # rep tweets w/ uncontested candidates removed
rep_tweets['party'] = [0 for i in range(len(rep_tweets))]

all_tweets = pd.read_csv('../data/tweet_data/tweets_with_topics.csv', index_col=False) # all tweets with labeled topics



In [127]:
all_tweets_no_topics = pd.read_csv('../data/tweet_data/all_tweets_cleaned.csv', index_col=False)


In [128]:
print(all_tweets_no_topics[all_tweets_no_topics['tweet_idx'] == 300000].stripped_text.values)
print(all_tweets[all_tweets['tweet_idx'] == 300000].stripped_text.values)

['our volunteers are are making calls for #gimmefitz all over the state. these knoxville supporters had a great time talking to their neighbors about our candidate fitz!']
['our volunteers are are making calls for #gimmefitz all over the state. these knoxville supporters had a great time talking to their neighbors about our candidate fitz!']


In [5]:
all_tweets.head(3)

,tweet_idx,stripped_text,party,gun_control,health_care,abortion,immigration,kavanaugh,education,jobs,tax,fake_news,environment,sexual_assault,gender,women
0,0,with corruption being one of the biggest conce...,0,False,0,0,0,0,0,0,0,0,0,False,False,False
1,1,with corruption being one of the biggest conce...,0,False,0,0,0,0,0,0,0,0,0,False,False,False
2,2,here is an important notice concerning christo...,0,False,0,0,0,0,0,0,0,0,0,False,False,False


In [97]:
gun_control_keywords = [' gun ', 'gun ', 'gun control', 'guncontrol', 'gun reform', 'gunreformnow', 'nra'
                           ,'2nd amendment',  'assault weapon', '2a', 'firearm', 'tree of life', 'columbine'
                            , 'marchforourlives', 'shotgun', 'second amendment', 'gunsense', 'gunsensenow',
                       'momsdemand', 'gunviolence', 'wearorange', 'gunreform', 'semi-automatic', 'assault weapon'
                       , 'assault-type weapons']

healthcare_keywords = ['healthcare', 'health care', 'medicare', 'medicaid', 'health-care', 'affordable care act', 'aca ']

abortion_keywords = ['prolife', 'pro-life', 'pro life', 'prochoice', 'pro-choice', 'pro choice', 'reproductive'
                     , 'abortion', 'roe45', ' roe ']

immigration_keywords = ['immigration', 'borders', 'secure border', 'build wall', 'build a wall', 'build the wall'
                       , 'buildthewall', 'immigrant']

kavanaugh_keywords = ['kavanaugh']
education_keywords = ['education']
jobs_keywords = ['jobs', 'wages', 'workers']
tax_keywords = ['taxes', 'tax cut']
fake_news_keywords = ['fake news']
environment_keywords = ['global warming', 'plastic pollution'
                       , 'renewable energy', 'earthday', 'coastline', 
                        'renewable', 'solar', 'climatechange', 'carbon', 'fossil'
                       , 'nuclear plant']
women_keywords = ['women\'s rights', 'sexual assault', 'metoo']
lgbt_keywords = ['lgbt', 'transgender']
daca_keywords = ['daca']
freedom_keywords = ['freedom']
trump_keywords = ['trump', 'the president', 'potus']

def label_topic(tweet, keywords):
    for kw in keywords:
        if kw in tweet:
            return 1
    return 0

def re_label_topic(df, topic, keywords):
    for i, r in df.iterrows():
        if r[topic] == 0:
            r[topic] = label_topic(r['stripped_text'], keywords)
    
# all_tweets['gun_control'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, gun_control_keywords))
# all_tweets['health_care'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, healthcare_keywords))
# all_tweets['abortion'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, abortion_keywords))
# all_tweets['immigration'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, immigration_keywords))
# all_tweets['kavanaugh'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, kavanaugh_keywords))
# all_tweets['education'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, education_keywords))
# all_tweets['jobs'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, jobs_keywords))
# all_tweets['tax'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, tax_keywords))
# all_tweets['fake_news'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, fake_news_keywords))
# all_tweets['environment'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, environment_keywords))
# all_tweets['women'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, women_keywords))
# all_tweets['lgbt'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, lgbt_keywords))
# all_tweets['freedom'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, freedom_keywords))
# all_tweets['trump'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, trump_keywords))


# for k in environment_keywords:
#     all_tweets.loc[all_tweets['stripped_text'].str.contains(k), 'environment'] = 1


In [98]:
# all_tweets = all_tweets.drop(labels=['trupm'], axis=1)
all_tweets.head(5)

,tweet_idx,stripped_text,party,gun_control,health_care,abortion,immigration,kavanaugh,education,jobs,tax,fake_news,environment,sexual_assault,gender,women,lgbt,freedom,trump
0,0,with corruption being one of the biggest conce...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0
1,1,with corruption being one of the biggest conce...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0
2,2,here is an important notice concerning christo...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0
3,3,i believe that we need to find real solutions ...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0
4,4,"it's time to turn the page on alabama's past, ...",0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0


# START HERE 

In [99]:
topic = 'trump'
# all_tweets = all_tweets_topics

print(len(all_tweets[all_tweets[topic] == 1]))
topic_tweets = pd.concat([all_tweets[all_tweets[topic] == 1], all_tweets[all_tweets[topic] == 0].sample(len(all_tweets[all_tweets[topic] == 1]))])
print(len(topic_tweets))
full_tweets = pd.concat([topic_tweets, all_tweets[~all_tweets['tweet_idx'].isin(topic_tweets['tweet_idx'].unique())]])

print(len(all_tweets))
print(len(full_tweets))
# 

35400
70800
557549
557549


In [100]:
for t in topic_tweets[topic_tweets[topic] == 1].sample(100)['stripped_text'].values:
    print(t, '\n')

feinstein said everyone pay their fair share" tax corps & wealthy, bring usa down i want to help @potus level the playing field by improving us & stability to rise usa up and raise the living standards of the world! which do you want? donate:  " 

rt to call on the gop to come to the negotiating table and work with democrats to protect #dreamers, fund #chip and community health centers, support veterans, make real investments in the fight against opioids and save americans pensions. end the #trumpshutdown and do your job! 

but this is the kind of chaos that can be expected as long as we keep sending the same people to washington who refuse to lead and refuse to hold president @realdonaldtrump in check. #trumpshutdown 

i took an oath in the secret service to step into the line of fire. as governor i will fight to turn oklahoma around and i will support and stand with president trump as he works to improve our economy. #lamb2018 #renewok 

the il gop is not backing away from trumpism. 

In [101]:
# tokenize data 

model_token_data = get_text_data(full_tweets['stripped_text'].values)
# dictionary, corpus = get_dict_corpus(model_token_data)
# print(len(dictionary))
X = [{w: 1 for w in x} for x in model_token_data]
print(X[0])
dv = DictVectorizer()
X = dv.fit_transform(X)

print(X.shape)
# print(X.todense())
names = dv.get_feature_names()

0 / 557549
20000 / 557549
40000 / 557549
60000 / 557549
80000 / 557549
100000 / 557549
120000 / 557549
140000 / 557549
160000 / 557549
180000 / 557549
200000 / 557549
220000 / 557549
240000 / 557549
260000 / 557549
280000 / 557549
300000 / 557549
320000 / 557549
340000 / 557549
360000 / 557549
380000 / 557549
400000 / 557549
420000 / 557549
440000 / 557549
460000 / 557549
480000 / 557549
500000 / 557549
520000 / 557549
540000 / 557549
finished
{'voter': 1, 'intimidation': 1, 'first': 1, 'candidate': 1, 'public': 1, 'office': 1, 'trump': 1, 'mywhy': 1, 'people': 1, 'alabama': 1, 'writemein': 1, 'governor': 1, 'rethinkalabama': 1, 'STOP': 1}
(557549, 122474)


In [104]:
print(len(topic_tweets))
topic_subset = X[:len(topic_tweets)]

X_train, X_test, y_train, y_test = train_test_split(topic_subset, topic_tweets[topic].values[:len(topic_tweets)], test_size=0.1)

new_file = True
if os.path.isfile('pickles/{}_logistic_regr.pkl'.format(topic)):
    with open('pickles/{}_logistic_regr.pkl'.format(topic), 'rb') as f:
        clf = pickle.load(f)
    new_file = False
else:
    clf = LogisticRegression()
    
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

# save model
with open('pickles/{}_logistic_regr.pkl'.format(topic), 'wb') as f:
    pickle.dump(clf, f)

for w, n in sorted(zip(clf.coef_[0], names), key=lambda e:e[0], reverse=True):
    print(w, n)

70800
10.81375856780444 trump
7.654092760277452 potus
6.520008219551635 trumptaxscam
6.3986620496718185 realdonaldtrump
6.241027433674397 trumpshutdown
5.620162469943262 mutetrump
5.55111497078301 trumptrain
5.535616848761098 trumprussia
5.3899451479556655 trumpcare
5.312452912874139 trumprally
5.205269794624264 president
5.060552079006104 trumpism
4.775302213209734 impeachtrump
4.6399126620529385 trumpbudget
4.631082821553472 trump2020
4.588613569331311 trumpwins
4.52238519061504 trumper
4.402483738434666 trumpian
4.343555063014379 trumpsellsout
4.204247141174205 presidential
4.018406725873289 trumpers
4.003071559879794 SCREEN_NAME
3.890093303352247 trumptariffs
3.8505579718648644 trumprallytn
3.7579771691976633 trumptradewar
3.631722255491081 latinosfortrump
3.5884168619769876 nevertrump
3.544459101032228 trumpcountry
3.5437941147352805 nevertrumper
3.5395056389728246 trumpet
3.418412696369116 donaldtrump
3.3260218828360233 trumpsters
3.3222586966762337 resisttrump
3.278954522855807 

0.618697539688619 putting
0.6184708731151519 rabies
0.6183719403651273 gleaming
0.6183278249113484 blocking
0.6181288524471747 coalesce
0.6180235537409301 cruel
0.6177915149632863 ariana
0.6170083111035334 trashygop
0.6166776310289424 notok
0.6163517886548309 desperately
0.6161972876223014 bergdahl
0.6161074411016593 trumpputinhelsinki
0.6160407979744783 qanon
0.6160273552211547 crash
0.6157636685582366 equate
0.6148635291269713 uncivil
0.6146695747508223 thankyoualabama
0.6136657327843105 herrera
0.6136446496169831 iraniandeal
0.6135390652464034 embrace
0.6134313409035966 red2bluein22
0.6134313409035966 ruchi
0.6134313409035966 stevebrownforcongress22
0.6124735125957333 istandwithisrael
0.6114733089103427 veteransresist
0.611177993626975 delaware
0.6110402258167741 turnberry
0.6109726109395566 cresent
0.6106817952000455 dominant
0.6106600399616942 councilor
0.6105710659905732 work
0.6103097458853289 slash
0.610028411571968 potus+house
0.6097069596888548 mcgahn
0.6096580196031901 socia

0.48122052346692695 electlong
0.4806936309487435 .@babetcohen
0.4806080473253493 pro2a
0.4805059057366964 poorperformance
0.4805059057366964 poorplanning
0.4805059057366964 poorprinciples
0.48029560856914244 victory
0.48019282630596477 truck
0.480009155138332 condemnation
0.4799385438326584 complicate
0.4798808444252479 substitute
0.4797821364158768 africanamericans
0.47952200709188325 cafeteria
0.4794770059779153 instability
0.47907380693816043 alinsky
0.47907380693816043 lucifer
0.47906594382436674 unsettle
0.478916913584835 untouchable
0.4788758915083926 cantwell
0.47849872896249585 1310am
0.4784917810100146 dictionary
0.4782516315757021 forgottenmenandwomen
0.4781677937674745 texasshooting
0.47786946218194076 naacpforum
0.47765946455513686 nomiddlemen
0.4775938692172907 nowar
0.47746437055409957 bigley
0.477462488258585 communication
0.4771072410063689 hideous
0.47700824563486205 erdogan
0.4766227819988039 count
0.47648737872905433 nationalprayerbreakfast
0.47642509197016303 lizard

0.38186976414732265 tennessee
0.38186552227894993 cagovdebate
0.3818242597176011 istandwithconservatism
0.3818089579838241 conspiracy
0.38175769230545925 blank
0.3816166885981358 nonsense
0.38141255608158264 jones
0.3814097311148696 clickbait
0.3813626996448623 irony
0.3813310494413863 grammar
0.3812321587189702 trumpery
0.38118736728809693 votemarsha
0.3808347006254473 lapse
0.3808177831579032 bipartisancare
0.38072648333601 pridemonth
0.3806974909886942 instruction
0.3806423342715092 instagram
0.38041965814598505 7et/6ct
0.38012339909027476 voteveteran
0.3800997557802788 custer
0.3800736489147617 sdvwayfair
0.3798790849064085 ability
0.3798504623199116 montemorris11
0.37980686141981806 lynch
0.3797725647702637 illness
0.3796791552166248 untrue
0.37958079332046046 strzok
0.3794860210779595 james
0.3794849305755839 violent
0.37947924657176996 shultz
0.37934639119359226 carcinoma!!=
0.37933288205878624 ignite
0.37884165478398785 patterson
0.3788315696376712 amigo
0.37877691821716114 ocb

0.32679484878007176 worry
0.3267662437396607 synchronize
0.32675487457313124 memoir
0.3266688818383064 gaming
0.3266280545188375 sarcastic
0.32649806838977097 everytime
0.32649736041645183 cactus
0.32649736041645183 southwestsplendor
0.3263180951094426 libby
0.326274165150722 voteantonio
0.3262313456140425 steelworker
0.326218310113454 calvert
0.32617057138186184 norway
0.3260918669045673 site
0.326045122047065 helsinki
0.32593330828961403 wibwdebate
0.3259124510115274 schoolteacher
0.3258869890529444 loophole
0.32585621258471886 fightingforyou
0.32583640239778566 7:30et/6:30
0.32574985476605595 marginal
0.3256660475596053 bean
0.3256099105646181 remark
0.32556034820439617 taxplan
0.3254173671536444 hurricanceflorence
0.3254061146328851 basket
0.32535516328680975 wareconomy
0.32531169605893606 russiaisnotafriend
0.32527869450315533 dangerous
0.3252327402666553 football
0.3251436800911685 choose
0.32510156066721746 castro
0.325059497783236 applause
0.325018711283961 barrett
0.3246484691

...

-0.2539306485336344 uhmmmmmmmmmm
-0.25394364581291967 linda
-0.2539823856133613 charitable
-0.25404120290530335 lausd
-0.2540741992653816 barlow
-0.2540902878066352 imwithkim
-0.2541049402355796 pragmatism
-0.25413264277902486 mlk50forward
-0.2541553330852947 bloodonherhands
-0.2541553330852947 killerkate
-0.2541553330852947 preborn
-0.2541553330852947 voteknute
-0.2541553330852947 voteknuteforgov
-0.2542154146874316 cardboard
-0.2542463763960162 mcguire
-0.2542912802973198 nrarussia
-0.25431259726824407 grandparent
-0.25433779623689473 teapartytom
-0.254436094230955 scar
-0.25454665209342503 uptons
-0.2545939456600602 itstime
-0.25461554784246215 incorporate
-0.25470047989463923 angela
-0.2547057348987449 machinery
-0.2547569608771989 plant
-0.2549358218466853 observation
-0.2549399135049536 stonemandouglasshooting
-0.2549594919905119 bloomberg
-0.254968278381625 kushner
-0.2549786245060237 heterosexuality
-0.2549786245060237 manhood
-0.2549786245060237 womanhood
-0.2550037757328465 b

-0.31592252762126083 pervert
-0.3159560962716831 modano
-0.3159560962716831 norwayne
-0.3161569623269257 kavanaghconfirmations
-0.316189133871626 coconut
-0.3162239367425444 renee
-0.31626248807469315 awareness
-0.31626279069618674 poverty
-0.31644598673167645 jewish
-0.3165446371845937 tngop
-0.3166123954151223 offshoring
-0.316626196843478 kempstrong
-0.3166875431956008 sands
-0.3167547699693927 educationforall
-0.31677377689272035 trip
-0.31698290359642906 square
-0.3170255120253124 party-
-0.31704409430131486 republicantaxplan
-0.31720504690051166 usher
-0.31740215986677783 mountaintop
-0.31753076914680317 polarplunge
-0.31753076914680317 takethat
-0.317622313914066 dilute
-0.3177586332069342 winfield
-0.3177640153215899 gretchens
-0.3177640153215899 winwithwhitmer
-0.31777476426253265 turk
-0.3178197427467158 cartoonist
-0.31800885787089694 intrinsic
-0.31803238362012654 outlet
-0.3180473151429068 5000match
-0.318117854258875 jammo
-0.3181489946050345 chrisbaker
-0.318208152195647

-0.3946190584699736 va02debate
-0.39463788249584575 holland
-0.3947026892343757 brigety
-0.3947026892343757 reuben
-0.39487271851074407 lifestyle
-0.3948959405588799 corruption-
-0.3949872411039397 rollback
-0.39503209348063223 kingwood
-0.39506141231708763 judas
-0.39506722999302224 nemesis
-0.3950901979142622 profit
-0.39523305357984273 employer
-0.39526547808125156 felony
-0.39542723127407264 superbowllll
-0.3956411169734802 safe
-0.3957536919609672 abdicate
-0.3957773943452821 canton
-0.39602753211880326 11:05
-0.39602753211880326 gebhart
-0.39624893956908536 cherness
-0.3962612519728624 --lee
-0.3963278562064937 tennesseetuesday
-0.39635435126560603 siegel
-0.39640252403548293 seldom
-0.39650183764355396 occasion
-0.3967855578742864 goverment
-0.3968020786750882 walkingnotworking
-0.39687499448632685 flipthe9thnc
-0.39715033161340785 teamdent
-0.3972072929574855 drudge
-0.39735626800506685 attraction
-0.39740705456336944 franklin
-0.3974555716836138 betteryourselfin3words
-0.39753

-0.5313883091452944 grateful
-0.5314719393471252 desperation
-0.5314909939915816 league
-0.5314916595119412 airline
-0.5320465999975588 bright
-0.5320600800169132 kates
-0.5322846552872073 66-years
-0.5323029494120325 monitoring
-0.5325082481155673 reentry
-0.5325707066209401 iam2018
-0.5325872930256409 monumentsforall
-0.5326020133056714 disavow
-0.5326710919898165 recall
-0.5327148890710786 encouragement
-0.5327996230623465 happymonday
-0.5329741145823976 library
-0.5334992124432323 comical
-0.5336732031682091 lebanon
-0.5336734772267889 arabia
-0.5336770964725216 relevant
-0.5339470310331625 ground
-0.5343421277825304 johnmccain
-0.5345239742956369 nomvp
-0.5345947586702212 environmentalist
-0.5346663319631796 planking
-0.5350160619396489 raft
-0.5355724054994315 menconi
-0.535614584890429 davidsmalling
-0.5358182418296877 uncertain
-0.5358398117978168 gutter
-0.5358570905812692 shocker
-0.5359425758868153 rule
-0.5361583902544571 extremism
-0.5367772995208444 flourish
-0.5369528543

-0.962603786629213 havoc
-0.9635869755190899 inclusive
-0.9636608673888887 gareth
-0.9653549352926474 martin
-0.9653721457364302 floor
-0.9656709543497333 truely
-0.9683095533668248 libertarian
-0.9694650570595962 civilize
-0.969705422463324 vladimir
-0.9705890185602281 kingston
-0.9710178832216358 boss
-0.9712770481744523 grandpa
-0.9718017747341615 practical
-0.9746210458519828 coastal
-0.9752123909162485 registration
-0.9753088082829495 panama
-0.9758922059167645 teamomar
-0.9760292227136615 nebraskavalues
-0.9788435058517677 yank
-0.9848031421854485 oliver
-0.9871005121189655 peoplefirst
-0.9872118844827494 commonwealth
-0.9877997078473008 profile
-0.9888173886707272 potato
-0.9888853378393103 radical
-0.9907262348323798 rebuilding
-0.9925377190104074 staten
-0.9930453971564305 inconvenient
-0.9936767010128547 silver
-0.9976290678165087 istandwithpp
-0.9976847442916178 remembering
-1.0015053512640415 teamblack
-1.0015526688349083 aggressor
-1.0063533580708957 renewables
-1.00796104

In [107]:
c = 0
for i in y_train:
    if i == 1:
        c += 1 
print(c)
print(len(y_train))

31858
63720


In [108]:
score

0.9149717514124294

## Run Predictions

In [109]:
pred_probs = clf.predict_proba(X[len(topic_tweets) + 1:])
pred_probs[0]

array([0.99824829, 0.00175171])

In [110]:
predictions = clf.predict(X[len(topic_tweets) + 1:])
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [111]:
c = 0
t = 0
tw = []
below_threshold = 0
tweets = full_tweets['stripped_text'].values[len(topic_tweets) + 1:]
full_tweets_idx = full_tweets['tweet_idx'].values[len(topic_tweets) + 1:]
idx = []
missed_tweets = []
tweets_above_threshold = []
for i in range(len(pred_probs)):
    if pred_probs[i][1] >= 0.99:
        t += 1
        idx.append(full_tweets_idx[i])
        tweets_above_threshold.append(tweets[i])
        
    else:
        below_threshold += 1
        
print("num above threshold: ", len(tweets_above_threshold)) # total above threshold
print("below threshold: ", below_threshold)

num above threshold:  884
below threshold:  485864


In [112]:
for t in np.random.choice(tweets_above_threshold, min(len(tweets_above_threshold), 100)):
    print(t, '\n')

leading mexican presidential candidate obrador calls for mass exodus to united states when he wins election   via @gatewaypundit 

the economy is booming and our #americafirst agenda is benefiting all  hispanic unemployment has now reached lowest level in 45 years! yet @susieleenv wants to reverse that historic achievement and make america a failure again. vote tark! #nv03  

jaime, sophia, i had a great time hearing vice president @mike_pence rally supporters today in carson city. isabella and jack would have too but it was nap time! #nvgov 

breaking: @politifact #lieofyear goes to president 45.joins ranks of pittenger."-cc #nc09 #flipthe9th #countryoverparty #staywoke #calledtoserveothers #staywoke #dantheflipper #wherewasdanin2016 #windmillgate @ourrevolution @latinovictoryus @votolatino   

. @joenbc dear mr. president, it has become obvious that you don't have an ivy league education either. you may have gone to an ivy league school, but you have demonstrated that you didn't lear

In [129]:
# label the tweets above threshold with a 1
# all_tweets.loc[all_tweets['tweet_idx'].isin(idx), topic] = 1

all_tweets.to_csv('../data/tweet_data/tweets_with_topics.csv', index=False)

In [569]:
all_tweets.head(10)

,tweet_idx,stripped_text,party,gun_control,health_care,abortion,immigration,kavanaugh,education,jobs,tax
0,0,with corruption being one of the biggest conce...,0,0,0,0,0,0,0,0,0
1,1,with corruption being one of the biggest conce...,0,0,0,0,0,0,0,0,0
2,2,here is an important notice concerning christo...,0,0,0,0,0,0,0,0,0
3,3,i believe that we need to find real solutions ...,0,0,0,0,0,0,0,0,0
4,4,"it's time to turn the page on alabama's past, ...",0,0,0,0,0,0,0,0,0
5,5,there is hope for alabama. this inspiring vide...,0,0,0,0,0,0,0,0,0
6,6,live,0,0,0,0,0,0,0,0,0
7,7,join us for a special and important webcast co...,0,0,0,0,0,0,0,0,0
8,8,countryman isn't standing down...his fight con...,0,0,0,0,0,0,0,0,0
9,9,please join us for a very special 2018 alabama...,0,0,0,0,0,0,0,0,0


In [8]:
topics = ['gun_control', 'health_care', 'abortion', 'immigration', 'kavanaugh', 'education', 'jobs', 'tax']

unlabeled_tweets = all_tweets[(all_tweets['gun_control'] == 0) 
           & (all_tweets['health_care'] == 0)
          & (all_tweets['abortion'] == 0)
          & (all_tweets['immigration'] == 0)
          & (all_tweets['kavanaugh'] == 0)
          & (all_tweets['education'] == 0)
          & (all_tweets['jobs'] == 0)
          & (all_tweets['tax'] == 0)]

In [18]:
print(len(unlabeled_tweets))
print(len(unlabeled_tweets) / len(all_tweets))
print('\n')
for t in unlabeled_tweets.sample(10)['stripped_text'].values:
    print(t, '\n')

485470
0.8707216764804528


its great to be a new mexican  especially during green chile roasting season! i stopped by some stands in hatch to get my fix of fresh roasted chile! an important tradition, growing chile employs thousands and brings in hundreds of millions in revenue each year. #nmpol 

to the 1,177 brothers who lost their lives on the uss arizona and the 2,403 sailors, soldiers, marines and civilians who died at pearl harbor 76 years ago today, america will never forget your ultimate sacrifice. 

the campaign has already begun. sign up at  if you'd like to help! appreciate the support! 

i was speaking metaphorically about loving my neighbor in practical ways. but if you're asking about the lambs in that photo (from my family's farm), i'll be honest. (you deserve honesty.) they are mutton. while i deeply respect vegetarianism, i don't practice it. 

lots of doors still to knock on in here the final 10 days until the primary. fun and humbling to visit with people face-to-fa

In [102]:
len(unlabeled_tweets[unlabeled_tweets['stripped_text'].str.contains('corruption')])

1702

In [103]:
for t in unlabeled_tweets[unlabeled_tweets['stripped_text'].str.contains('corruption')].sample(100)['stripped_text'].values:
    print(t, '\n')

thank you to senator pierre from haiti and the delegation for their invitation today at the rotary club of bellaire. haiti is on the rise as they strive to eliminate corruption and create opportunity for its people. join our #armyofdavids!   

heartbreaking: sudan, world's last male northern white rhino, dies"  we must stop the multi-billion dollar industry that is wildlife trafficking. this industry thrives on corruption, funds terrorism, and results in the extinction of animals. @humanesociety" 

#crushcorruption #firefeinstein   

yeah there might be some major corruption going on here(?) 

my opponent @repmimiwalters sits on the house energy & commerce oversight committee, which has jurisdiction over jared & ivanka's alleged corruption. not only has she not called for a hearing, she's never once even publicly mentioned these disturbing reports. #wheresmimi 

joining @thedaveweinbaum today on his show! 11:15 pst we are gonna talk alabama corruption and election fraud!.. his link is 

In [32]:
len(all_tweets[all_tweets['abortion'] == 1])

2879